In [1]:
import pandas as pd
import datetime

## Create the dataset about inverters from Prometheus

*The dates must be changed in two places.*

In [2]:
#pip install prometheus-pandas

In [3]:
from prometheus_pandas import query

In [4]:
p = query.Prometheus('http://10.16.27.31:9090')

In [5]:
start = '2024-06-07T00:00:00Z'
end = '2024-06-26T00:00:00Z'

dates = [start]

date_now = start

while date_now!=end :
    date_datetime = datetime.datetime.strptime(date_now, "%Y-%m-%dT%H:%M:%SZ")
    date_now = str(date_datetime + datetime.timedelta(1,0)).replace(' ','T')+'Z'
    dates.append(date_now)

In [6]:
dates

['2024-06-07T00:00:00Z',
 '2024-06-08T00:00:00Z',
 '2024-06-09T00:00:00Z',
 '2024-06-10T00:00:00Z',
 '2024-06-11T00:00:00Z',
 '2024-06-12T00:00:00Z',
 '2024-06-13T00:00:00Z',
 '2024-06-14T00:00:00Z',
 '2024-06-15T00:00:00Z',
 '2024-06-16T00:00:00Z',
 '2024-06-17T00:00:00Z',
 '2024-06-18T00:00:00Z',
 '2024-06-19T00:00:00Z',
 '2024-06-20T00:00:00Z',
 '2024-06-21T00:00:00Z',
 '2024-06-22T00:00:00Z',
 '2024-06-23T00:00:00Z',
 '2024-06-24T00:00:00Z',
 '2024-06-25T00:00:00Z',
 '2024-06-26T00:00:00Z']

In [7]:
data = p.query_range('cyprus__nicosia__ucy__dc1__fronius_fronius_inverter_output_W_average{job="netdata"}',
        '2024-06-07T00:00:00Z', '2024-06-08T00:00:00Z', '15s')

In [8]:
for i in range(1,len(dates)-1) :
    data_to_append = p.query_range('cyprus__nicosia__ucy__dc1__fronius_fronius_inverter_output_W_average{job="netdata"}',
        dates[i], dates[i+1], '15s')
    data = pd.concat([data,data_to_append])

In [9]:
data

,"cyprus__nicosia__ucy__dc1__fronius_fronius_inverter_output_W_average{chart=""fronius_GetPowerFlowRealtimeData.inverters.output"",dimension=""inverter_1"",family=""inverters"",instance=""10.16.27.31:19998"",job=""netdata""}","cyprus__nicosia__ucy__dc1__fronius_fronius_inverter_output_W_average{chart=""fronius_GetPowerFlowRealtimeData.inverters.output"",dimension=""inverter_2"",family=""inverters"",instance=""10.16.27.31:19998"",job=""netdata""}","cyprus__nicosia__ucy__dc1__fronius_fronius_inverter_output_W_average{chart=""fronius_GetPowerFlowRealtimeData.inverters.output"",dimension=""inverter_3"",family=""inverters"",instance=""10.16.27.31:19998"",job=""netdata""}"
2024-06-07 00:50:15,NaN,0.0,NaN
2024-06-07 00:50:30,NaN,0.0,NaN
2024-06-07 00:50:45,NaN,0.0,NaN
2024-06-07 00:51:00,NaN,0.0,NaN
2024-06-07 00:51:15,NaN,0.0,NaN
...,...,...,...
2024-06-25 23:52:00,NaN,NaN,0.0
2024-06-25 23:52:15,NaN,NaN,0.0
2024-06-25 23:52:30,NaN,NaN,0.0
2024-06-25 23:52:45,NaN,NaN,0.0


In [10]:
data.rename({'cyprus__nicosia__ucy__dc1__fronius_fronius_inverter_output_W_average{chart="fronius_GetPowerFlowRealtimeData.inverters.output",dimension="inverter_1",family="inverters",instance="10.16.27.31:19998",job="netdata"}':'inverter1'},inplace=True,axis=1)
data.rename({'cyprus__nicosia__ucy__dc1__fronius_fronius_inverter_output_W_average{chart="fronius_GetPowerFlowRealtimeData.inverters.output",dimension="inverter_2",family="inverters",instance="10.16.27.31:19998",job="netdata"}':'inverter2'},inplace=True,axis=1)
data.rename({'cyprus__nicosia__ucy__dc1__fronius_fronius_inverter_output_W_average{chart="fronius_GetPowerFlowRealtimeData.inverters.output",dimension="inverter_3",family="inverters",instance="10.16.27.31:19998",job="netdata"}':'inverter3'},inplace=True,axis=1)

In [11]:
data = data.reset_index().drop_duplicates(subset='index', keep='first')

In [12]:
data.rename({'index':'date'},inplace=True,axis=1)

In [13]:
data

,date,inverter1,inverter2,inverter3
0,2024-06-07 00:50:15,NaN,0.0,NaN
1,2024-06-07 00:50:30,NaN,0.0,NaN
2,2024-06-07 00:50:45,NaN,0.0,NaN
3,2024-06-07 00:51:00,NaN,0.0,NaN
4,2024-06-07 00:51:15,NaN,0.0,NaN
...,...,...,...,...
78208,2024-06-25 23:52:00,NaN,NaN,0.0
78209,2024-06-25 23:52:15,NaN,NaN,0.0
78210,2024-06-25 23:52:30,NaN,NaN,0.0
78211,2024-06-25 23:52:45,NaN,NaN,0.0


In [14]:
data.to_csv('inverters_07.06.2024-25.06.2024.csv',index=False)

In [15]:
data['inverter3'].value_counts()

inverter3
0.0       12749
314.0        70
312.0        68
319.0        63
334.0        62
          ...  
7062.0        1
3521.0        1
3564.0        1
3979.0        1
1.0           1
Name: count, Length: 7080, dtype: int64